In [1]:
from __future__ import division
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from statannot import add_stat_annotation
from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

import time

import scipy

from sklearn.svm import SVR

from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.multitest import fdrcorrection

from statsmodels.stats import multitest

from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.ensemble import ExtraTreesClassifier

from collections import Counter
import heapq

import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor

import re

from sklearn.preprocessing import MinMaxScaler

import os

In [2]:
aper_df =  pd.read_excel('Data/aperiodic.xlsx', index_col=0)
aper_df



,Slope_HPC_left,Slope_HPC_right,Slope_CING_left,Slope_CING_right,Slope_TEMP_left,Slope_TEMP_right,Slope_PARIET_left,Slope_PARIET_right,Slope_OCC_left,Slope_OCC_right,...,Offset_PARIET_left,Offset_PARIET_right,Offset_OCC_left,Offset_OCC_right,Age,Sex,Educ,MMSE,ODQ,GINI
0,2.859768,1.767303,2.415282,2.727924,3.081733,2.077333,2.413411,1.768170,3.129080,2.081708,...,4.966953,4.076689,5.791904,4.230146,33.0,1.0,19.0,30.0,99.282421,40.70
1,1.309688,1.667206,1.528242,1.537064,1.376505,2.009045,1.636336,1.460334,1.433874,1.607140,...,3.199684,2.893094,2.758251,2.797152,57.0,2.0,13.0,NaN,99.739738,40.70
2,1.257444,0.840704,1.330353,1.294367,1.342255,0.898285,1.438093,1.368015,1.307053,1.185280,...,3.189643,3.151721,2.838725,2.506678,32.0,1.0,10.0,NaN,99.361739,40.70
3,1.682139,1.355977,1.080644,0.955786,1.684404,1.144267,1.448098,1.381354,1.495415,1.232343,...,3.314643,3.166422,3.164481,2.732750,21.0,1.0,13.0,NaN,99.561273,40.70
4,0.866675,0.366260,0.455986,0.425098,0.870994,0.353314,0.830158,0.652646,0.700754,0.562175,...,2.853273,2.634403,2.520505,2.308816,19.0,1.0,13.0,30.0,99.822774,40.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,0.860623,0.726887,0.984260,0.996106,0.811934,0.623448,1.105042,1.070750,1.269507,1.034332,...,2.562506,2.534879,2.678311,2.442810,57.0,2.0,17.0,30.0,98.650235,44.92
1444,3.240214,2.554994,4.075704,4.236230,2.822616,2.044794,3.631432,4.065509,3.820450,3.716575,...,6.439969,7.285327,6.601294,6.505917,48.0,2.0,17.0,28.0,99.599221,44.92
1445,2.732930,2.810929,2.014831,2.376631,2.349316,2.389446,1.987368,2.083951,4.578028,3.735156,...,3.964125,4.146706,8.038794,6.603005,57.0,2.0,17.0,29.0,99.675799,44.92
1446,3.003057,2.705750,3.486568,3.536880,2.642035,2.560009,3.293006,3.629184,3.510047,3.379987,...,6.010047,6.883034,6.353391,6.108625,69.0,2.0,12.0,30.0,99.641446,44.92


## Statistical significance check

In [3]:
outcomes_list = list(aper_df.iloc[:,0:-6])
df_metrics_slopes = pd.DataFrame(columns=outcomes_list)
df_metrics_pval = pd.DataFrame(columns=outcomes_list)
df_metrics_dir = pd.DataFrame(columns=outcomes_list)
df_metrics_slopes

,Slope_HPC_left,Slope_HPC_right,Slope_CING_left,Slope_CING_right,Slope_TEMP_left,Slope_TEMP_right,Slope_PARIET_left,Slope_PARIET_right,Slope_OCC_left,Slope_OCC_right,...,Offset_HPC_left,Offset_HPC_right,Offset_CING_left,Offset_CING_right,Offset_TEMP_left,Offset_TEMP_right,Offset_PARIET_left,Offset_PARIET_right,Offset_OCC_left,Offset_OCC_right


In [4]:
from scipy.stats import linregress

df_metrics_slopes = pd.DataFrame(columns=outcomes_list)

for variable_y in outcomes_list:
    
    c_aper_df_mmse = aper_df[['GINI', variable_y]]
    
    c_aper_df_mmse = c_aper_df_mmse.dropna()
    c_aper_df_mmse.reset_index(drop=True)
    
    y_conn_1A = c_aper_df_mmse[variable_y]
    X_conn_1A = c_aper_df_mmse[['GINI']]
    
    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_conn_1A)
    X_conn_1A = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)

    X_conn_1A =  X_conn_1A.copy()

    slope, intercept, r_value, p_value, std_err = linregress(X_conn_1A['GINI'], y_conn_1A)
    
    p_value*=210
    if(p_value<=0.05):
        df_metrics_slopes.loc['GINI', variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' 
    
    df_metrics_pval.loc['GINI', variable_y] =  p_value
    df_metrics_dir.loc['GINI', variable_y] =  slope
    
for variable_y in outcomes_list:
    for variable_x in ['Age', 'Educ', 'Sex', 'MMSE']:
    
        c_aper_df_mmse = aper_df[['GINI', variable_x, variable_y]]

        c_aper_df_mmse = c_aper_df_mmse.dropna()
        c_aper_df_mmse.reset_index(drop=True)
        
        y_conn_1A = c_aper_df_mmse[variable_y]
        X_conn_1A = c_aper_df_mmse[['GINI', variable_x]]
        
        scaler = MinMaxScaler([0.05, 0.95])
        scaling_data = scaler.fit_transform(X_conn_1A)
        X_conn_1A = pd.DataFrame(scaling_data, columns= X_conn_1A.columns, index = X_conn_1A.index)

        X_conn_1A =  X_conn_1A.copy()

        slope, intercept, r_value, p_value, std_err = linregress(X_conn_1A[variable_x],y_conn_1A)
    
        p_value*=210
        if(p_value<=0.05):
            df_metrics_slopes.loc[variable_x, variable_y] =  str(np.round(slope,3)) + ' (' + str(np.round(p_value,3))+')' 
     
        df_metrics_pval.loc[variable_x, variable_y] =  p_value
        df_metrics_dir.loc[variable_x, variable_y] =  slope

In [5]:
df_metrics_slopes = df_metrics_slopes.T
df_metrics_pval = df_metrics_pval.T
df_metrics_dir = df_metrics_dir.T
df_metrics_dir

,GINI,Age,Educ,Sex,MMSE
Slope_HPC_left,-0.905006,-0.253507,0.031171,-0.179794,1.025307
Slope_HPC_right,-0.752266,-0.329491,0.17723,-0.163585,0.942284
Slope_CING_left,-0.965606,-0.722428,0.668924,-0.180915,1.140162
Slope_CING_right,-1.05444,-0.764066,0.831233,-0.225226,1.12009
Slope_TEMP_left,-0.940845,-0.304356,-0.435535,-0.24448,0.923727
Slope_TEMP_right,-0.756375,-0.352368,-0.392504,-0.247511,0.988496
Slope_PARIET_left,-0.925928,-0.913985,0.456058,-0.205978,1.021998
Slope_PARIET_right,-0.892486,-0.900559,0.626909,-0.146254,1.081639
Slope_OCC_left,-0.926153,-0.35525,-0.382908,-0.128652,0.913233
Slope_OCC_right,-0.816708,-0.558228,-0.323291,-0.161429,0.992483


In [6]:
df_metrics_GINI = df_metrics_slopes[['GINI']]
df_metrics_GINI = df_metrics_GINI.copy()
df_metrics_GINI.dropna(inplace=True)

In [7]:
outcomes_list = list(df_metrics_GINI.index)
outcomes_list

['Slope_HPC_left',
 'Slope_HPC_right',
 'Slope_CING_left',
 'Slope_CING_right',
 'Slope_TEMP_left',
 'Slope_TEMP_right',
 'Slope_PARIET_left',
 'Slope_PARIET_right',
 'Slope_OCC_left',
 'Slope_OCC_right',
 'Offset_HPC_left',
 'Offset_HPC_right',
 'Offset_CING_left',
 'Offset_CING_right',
 'Offset_TEMP_left',
 'Offset_TEMP_right',
 'Offset_PARIET_left',
 'Offset_PARIET_right',
 'Offset_OCC_left']

In [8]:
def classifier_ROC(X_, y, fig_show = True, params_ = None):
    
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=10000, stratify=y)

    #lista = list(sfs_.k_feature_names_)

    #X_train = X_train[lista]
    #X_test = X_test[lista]

    cv    = RepeatedKFold(n_splits=10, n_repeats=2, random_state=101)
    folds = [(train,test) for train, test in cv.split(X_train, y_train)]
    metrics = ['auc', 'fpr', 'tpr', 'thresholds', 'f1_score', 'accuracy_score', 'recall_score', 'precision_score', 'confusion_matrix', 'model']
    results = {
        'train': {m:[] for m in metrics},
        'val'  : {m:[] for m in metrics},
        'test' : {m:[] for m in metrics}
    }
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)


    if(params_!=None):
        params = params_
    else:
        params = {
            'objective'   : 'binary:logistic',
            'eval_metric' : 'logloss'
        }


    plt.rcParams["figure.figsize"] = (10,10)
    dtest = xgb.DMatrix(X_test, label=y_test)
    for train, test in tqdm(folds, total=len(folds)):
        dtrain = xgb.DMatrix(X_train.iloc[train,:], label=y_train.iloc[train])
        dval   = xgb.DMatrix(X_train.iloc[test,:], label=y_train.iloc[test])
        model  = xgb.train(
            dtrain                = dtrain,
            params                = params, 
            evals                 = [(dtrain, 'train'), (dval, 'val')],
            num_boost_round       = 1000,
            verbose_eval          = False,
            early_stopping_rounds = 10,
        )
        sets = [dtrain, dval, dtest]

        for i,ds in enumerate(results.keys()):
            y_preds              = model.predict(sets[i])
            labels               = sets[i].get_label()
            fpr, tpr, thresholds = roc_curve(labels, y_preds)
            results[ds]['fpr'].append(fpr)
            results[ds]['tpr'].append(tpr)
            results[ds]['thresholds'].append(thresholds)
            results[ds]['auc'].append(roc_auc_score(labels, y_preds))

            results[ds]['f1_score'].append(f1_score(labels, np.round(y_preds)))
            results[ds]['accuracy_score'].append(accuracy_score(labels, np.round(y_preds)))
            results[ds]['recall_score'].append(recall_score(labels, np.round(y_preds)))
            results[ds]['precision_score'].append(precision_score(labels, np.round(y_preds)))
            results[ds]['confusion_matrix'].append(confusion_matrix(labels, np.round(y_preds)))
            results[ds]['model'].append(model)


    kind = 'test'
    c_fill      = 'rgba(52, 152, 219, 0.2)'
    c_line      = 'rgba(52, 152, 219, 0.5)'
    c_line_main = 'rgba(41, 128, 185, 1.0)'
    c_grid      = 'rgba(189, 195, 199, 0.5)'
    c_annot     = 'rgba(149, 165, 166, 0.5)'
    c_highlight = 'rgba(192, 57, 43, 1.0)'

    fpr_mean    = np.linspace(0, 1, 100)
    interp_tprs = []

    for i in range(10*2):
        fpr           = results[kind]['fpr'][i]
        tpr           = results[kind]['tpr'][i]
        interp_tpr    = np.interp(fpr_mean, fpr, tpr)
        interp_tpr[0] = 0.0
        interp_tprs.append(interp_tpr)

    tpr_mean     = np.mean(interp_tprs, axis=0)
    tpr_mean[-1] = 1.0
    tpr_std      = 2*np.std(interp_tprs, axis=0)
    tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
    tpr_lower    = tpr_mean-tpr_std
    auc          = np.mean(results[kind]['auc'])
    
    roc_dict = {}
    roc_dict['tpr_mean'] = tpr_mean
    roc_dict['tpr_std'] = tpr_std
    roc_dict['tpr_upper'] = tpr_upper
    roc_dict['tpr_lower'] = tpr_lower
    roc_dict['auc'] = auc

    if(fig_show):
        fig = go.Figure([
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_upper,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'upper'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_lower,
                fill       = 'tonexty',
                fillcolor  = c_fill,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'lower'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_mean,
                line       = dict(color=c_line_main, width=2),
                hoverinfo  = "skip",
                showlegend = True,
                name       = f'AUC: {auc:.3f}')
        ])

        fig.add_shape(
            type ='line', 
            line =dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
        )

        fig.update_layout(
            template    = 'plotly_white', 
            title_x     = 0.5,
            xaxis_title = "1 - Specificity",
            yaxis_title = "Sensitivity",
            width       = 800,
            height      = 800,
            legend      = dict(
                yanchor="bottom", 
                xanchor="right", 
                x=0.95,
                y=0.01,
            )
        )

        fig.update_yaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            scaleanchor = "x", 
            scaleratio  = 1,
            linecolor   = 'black')

        fig.update_xaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            constrain   = 'domain',
            linecolor   = 'black')



        fig.show()
        
    else:
        print(y.name, 'AUC: ', np.round(auc,2), '+-( ', np.round(np.std(results[kind]['auc']),2), ')', X_.shape)

    return (model, results, np.round(auc,2), roc_dict)

In [9]:
from skopt import BayesSearchCV
def SearchBestModel(case_x, case_y):

    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.2, random_state=1)

    opt_XGB = BayesSearchCV(
            XGBClassifier(),
            {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            },
                          
            #n_iter=10,
            #cv=3,
            random_state=1
        )

    opt_XGB.fit(X_train, y_train)


    print('XGB')
    print("val. score: %s" % opt_XGB.best_score_)
    print("test score: %s" % opt_XGB.score(X_test, y_test))
    print("best parameters: %s" % str(opt_XGB.best_params_))
    print('---------------------------------------------\n')
    
    return opt_XGB

## Classification:  Gini

In [10]:
outcomes_list = list(aper_df.iloc[:,0:-7].columns)
gini_results_preliminar = {}
gini_results_auc_preliminar = {}
for outcome in outcomes_list:

    aper_df_mmse = aper_df[[outcome, 'GINI']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()
    
    try:
        r = classifier_ROC(X_aper_1A, y_aper_1A, False)
        gini_results_preliminar[outcome] = r
        gini_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

Slope_HPC_left_b AUC:  0.73 +-(  0.0 ) (1374, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

Slope_HPC_right_b AUC:  0.68 +-(  0.0 ) (1373, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

...

Offset_PARIET_left_b AUC:  0.79 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

Offset_PARIET_right_b AUC:  0.79 +-(  0.0 ) (1376, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

Offset_OCC_left_b AUC:  0.75 +-(  0.0 ) (1376, 1)


In [11]:
top_10_keys_GINI_preliminar = heapq.nlargest(10, gini_results_auc_preliminar, key=gini_results_auc_preliminar.get)
top_10_keys_GINI_preliminar

['Offset_HPC_left',
 'Offset_PARIET_left',
 'Offset_PARIET_right',
 'Offset_HPC_right',
 'Offset_TEMP_left',
 'Slope_TEMP_left',
 'Slope_PARIET_left',
 'Offset_TEMP_right',
 'Knee_TEMP_right',
 'Offset_OCC_left']

In [12]:
gini_results = {}
gini_results_auc = {}
gini_paragram = {}
for outcome in top_10_keys_GINI_preliminar:

    aper_df_mmse = aper_df[[outcome, 'GINI']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_aper_1A, y_aper_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_aper_1A, y_aper_1A, False, params_)
    gini_results[outcome] = r
    gini_results_auc[outcome] = r[2]
    gini_paragram[outcome] = params_

XGB
val. score: 0.7834786218347862
test score: 0.7927272727272727
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'auc'), ('learning_rate', 0.2), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.1), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Offset_HPC_left_b AUC:  0.81 +-(  0.0 ) (1374, 1)
XGB
val. score: 0.6654545454545455
test score: 0.677536231884058
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'aucpr'), ('learning_rate', 0.2), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 1.0), ('subsample', 0.8)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Offset_PARIET_left_b AUC:  0.78 +-(  0.0 ) (1376, 1)
XGB
val. score: 0.6590909090909092
test score: 0.6956521739130435
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'error'), ('learning_rate', 0.01), ('max_depth', 5), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 0.1), ('subsample', 1.0)])
---------------------------------------------



...

Offset_TEMP_right_b AUC:  0.77 +-(  0.01 ) (1376, 1)
XGB
val. score: 0.730909090909091
test score: 0.7971014492753623
best parameters: OrderedDict([('colsample_bytree', 0.9), ('eval_metric', 'auc'), ('learning_rate', 0.1), ('max_depth', 3), ('objective', 'binary:logistic'), ('reg_alpha', 1.0), ('reg_lambda', 0.0), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Knee_TEMP_right_b AUC:  0.74 +-(  0.01 ) (1376, 1)
XGB
val. score: 0.7318181818181818
test score: 0.7391304347826086
best parameters: OrderedDict([('colsample_bytree', 1.0), ('eval_metric', 'auc'), ('learning_rate', 0.2), ('max_depth', 4), ('objective', 'binary:logistic'), ('reg_alpha', 0.1), ('reg_lambda', 0.1), ('subsample', 0.9)])
---------------------------------------------



  0%|          | 0/20 [00:00<?, ?it/s]

Offset_OCC_left_b AUC:  0.75 +-(  0.0 ) (1376, 1)


In [13]:
top_10_keys_GINI = heapq.nlargest(10, gini_results_auc, key=gini_results_auc.get)
top_10_keys_GINI

['Offset_HPC_left',
 'Offset_PARIET_right',
 'Offset_PARIET_left',
 'Offset_HPC_right',
 'Offset_TEMP_left',
 'Slope_TEMP_left',
 'Slope_PARIET_left',
 'Offset_TEMP_right',
 'Offset_OCC_left',
 'Knee_TEMP_right']

## Classification:  Gini + Age

In [ ]:
outcomes_list = list(aper_df.iloc[:,0:-7].columns)
gini_age_results_preliminar = {}
gini_age_results_auc_preliminar = {}
for outcome in outcomes_list:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'Age']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()
    
    try:
        r = classifier_ROC(X_aper_1A, y_aper_1A, False)
        gini_age_results_preliminar[outcome] = r
        gini_age_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

  0%|          | 0/20 [00:00<?, ?it/s]

Slope_HPC_left_b AUC:  0.73 +-(  0.01 ) (1374, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

Slope_HPC_right_b AUC:  0.69 +-(  0.01 ) (1373, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Knee_PARIET_right_b AUC:  0.7 +-(  0.01 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

Knee_OCC_left_b AUC:  0.68 +-(  0.01 ) (1376, 2)


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
top_10_keys_GINI_AGE_preliminar = heapq.nlargest(10, gini_age_results_auc_preliminar, key=gini_age_results_auc_preliminar.get)

In [ ]:
gini_age_results = {}
gini_age_results_auc = {}
gini_age_paragram = {}
for outcome in top_10_keys_GINI_AGE_preliminar:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'Age']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'Age']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_aper_1A, y_aper_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_aper_1A, y_aper_1A, False, params_)
    gini_age_results[outcome] = r
    gini_age_results_auc[outcome] = r[2]
    gini_age_paragram[outcome] = params_

In [ ]:
top_10_keys_GINI_age = heapq.nlargest(10, gini_age_results_auc, key=gini_age_results_auc.get)
top_10_keys_GINI_age

## Classification:  Gini + Educ

In [ ]:
## Classification:  Gini + Educ

outcomes_list = list(aper_df.iloc[:,0:-7].columns)
gini_educ_results_preliminar = {}
gini_educ_results_auc_preliminar = {}
for outcome in outcomes_list:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'Educ']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()
    
    try:
        r = classifier_ROC(X_aper_1A, y_aper_1A, False)
        gini_educ_results_preliminar[outcome] = r
        gini_educ_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

In [ ]:
top_10_keys_GINI_educ_preliminar = heapq.nlargest(10, gini_educ_results_auc_preliminar, key=gini_educ_results_auc_preliminar.get)

In [ ]:
gini_educ_results = {}
gini_educ_results_auc = {}
gini_educ_paragram = {}
for outcome in top_10_keys_GINI_educ_preliminar:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'Educ']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'Educ']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_aper_1A, y_aper_1A)
    params_ = dict(opt_XGB.best_params_)


    r = classifier_ROC(X_aper_1A, y_aper_1A, False, params_)
    gini_educ_results[outcome] = r
    gini_educ_results_auc[outcome] = r[2]
    gini_educ_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_educ = heapq.nlargest(10, gini_educ_results_auc, key=gini_educ_results_auc.get)
top_10_keys_gini_educ

## Classification:  Gini + Sex

In [ ]:
outcomes_list = list(aper_df.iloc[:,0:-7].columns)
gini_sex_results_preliminar = {}
gini_sex_results_auc_preliminar = {}
for outcome in outcomes_list:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'Sex']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()
    
    try:
        r = classifier_ROC(X_aper_1A, y_aper_1A, False)
        gini_sex_results_preliminar[outcome] = r
        gini_sex_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

In [ ]:
top_10_keys_GINI_sex_preliminar = heapq.nlargest(10, gini_sex_results_auc_preliminar, key=gini_sex_results_auc_preliminar.get)

In [ ]:
gini_sex_results = {}
gini_sex_results_auc = {}
gini_sex_paragram = {}
for outcome in top_10_keys_GINI_sex_preliminar:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'Sex']]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'Sex']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_aper_1A, y_aper_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_aper_1A, y_aper_1A, False, params_)
    gini_sex_results[outcome] = r
    gini_sex_results_auc[outcome] = r[2]
    gini_sex_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_sex = heapq.nlargest(10, gini_sex_results_auc, key=gini_sex_results_auc.get)
top_10_keys_gini_sex

## Classification:  Gini + Cognition

In [ ]:
## Classification:  Gini + Cognition

outcomes_list = list(aper_df.iloc[:,0:-7].columns)
gini_mmse_results_preliminar = {}
gini_mmse_results_auc_preliminar = {}
for outcome in outcomes_list:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'MMSE']]
    aper_df_mmse = aper_df_mmse[aper_df_mmse['MMSE'] >= 21]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()
    
    try:
        r = classifier_ROC(X_aper_1A, y_aper_1A, False)
        gini_mmse_results_preliminar[outcome] = r
        gini_mmse_results_auc_preliminar[outcome] = r[2]
    except:
        print('Bad: ',outcome + '_b')

In [ ]:
top_10_keys_GINI_mmse_preliminar = heapq.nlargest(10, gini_mmse_results_auc_preliminar, key=gini_mmse_results_auc_preliminar.get)

In [ ]:
gini_mmse_results = {}
gini_mmse_results_auc = {}
gini_mmse_paragram = {}
for outcome in top_10_keys_GINI_mmse_preliminar:

    aper_df_mmse = aper_df[[outcome, 'GINI', 'MMSE']]
    aper_df_mmse = aper_df_mmse[aper_df_mmse['MMSE'] >= 21]
    aper_df_mmse = aper_df_mmse.dropna()
    aper_df_mmse.reset_index(drop=True)

   
    mediana = aper_df_mmse[outcome].median()
    aper_df_mmse[outcome + '_b'] = (aper_df_mmse[outcome] > mediana).astype(int)
    
    y_aper_1A = aper_df_mmse[outcome + '_b']
    X_aper_1A = aper_df_mmse[['GINI', 'MMSE']]

    scaler = MinMaxScaler([0.05, 0.95])
    scaling_data = scaler.fit_transform(X_aper_1A)
    X_aper_1A_s = pd.DataFrame(scaling_data, columns= X_aper_1A.columns, index = X_aper_1A.index)
    
    X_aper_1A =  X_aper_1A_s.copy()

    
    opt_XGB = SearchBestModel(X_aper_1A, y_aper_1A)
    params_ = dict(opt_XGB.best_params_)
    

    r = classifier_ROC(X_aper_1A, y_aper_1A, False, params_)
    gini_mmse_results[outcome] = r
    gini_mmse_results_auc[outcome] = r[2]
    gini_mmse_paragram[outcome] = params_

In [ ]:
top_10_keys_gini_mmse = heapq.nlargest(10, gini_mmse_results_auc, key=gini_mmse_results_auc.get)
top_10_keys_gini_mmse